In [1]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from types import SimpleNamespace as SN
from IPython.core.display import Image, display
import os
import sys

np.random.seed(14)

In [2]:
# Set the checkpoint path

path = '/Users/mika/Downloads/smac_arxiv_coma_qmix_models/brown/qmix__8m__2018-11-30_04-55-00'

timestep = '10000807'

# Avalibale timesteps
# 237
# 2000386
# 4000528
# 6000674
# 8000769
# 10000787/

In [3]:
# Copied from refactor

class QMixer(nn.Module):
    def __init__(self, args):
        super(QMixer, self).__init__()

        self.args = args
        self.n_agents = args.n_agents
        self.state_dim = int(np.prod(args.state_shape))

        self.embed_dim = args.mixing_embed_dim

        self.hyper_w_1 = nn.Linear(self.state_dim, self.embed_dim * self.n_agents)
        self.hyper_w_final = nn.Linear(self.state_dim, self.embed_dim)

        # Initialise the hyper networks with a fixed variance, if specified
        if self.args.hyper_initialization_nonzeros > 0:
            std = self.args.hyper_initialization_nonzeros ** -0.5
            self.hyper_w_1.weight.data.normal_(std=std)
            self.hyper_w_1.bias.data.normal_(std=std)
            self.hyper_w_final.weight.data.normal_(std=std)
            self.hyper_w_final.bias.data.normal_(std=std)

        # Initialise the hyper-network of the skip-connections, such that the result is close to VDN
        if self.args.skip_connections:
            self.skip_connections = nn.Linear(self.state_dim, self.args.n_agents, bias=True)
            self.skip_connections.bias.data.fill_(1.0)  # bias produces initial VDN weights

        # State dependent bias for hidden layer
        self.hyper_b_1 = nn.Linear(self.state_dim, self.embed_dim)

        # V(s) instead of a bias for the last layers
        self.V = nn.Sequential(nn.Linear(self.state_dim, self.embed_dim),
                               nn.ReLU(),
                               nn.Linear(self.embed_dim, 1))

    def forward(self, agent_qs, states):
        bs = agent_qs.size(0)
        states = states.reshape(-1, self.state_dim)
        agent_qs = agent_qs.view(-1, 1, self.n_agents)
        # First layer
        w1 = th.abs(self.hyper_w_1(states))
        b1 = self.hyper_b_1(states)
        w1 = w1.view(-1, self.n_agents, self.embed_dim)
        b1 = b1.view(-1, 1, self.embed_dim)
        hidden = F.elu(th.bmm(agent_qs, w1) + b1)
        # Second layer
        w_final = th.abs(self.hyper_w_final(states))
        w_final = w_final.view(-1, self.embed_dim, 1)
        # State-dependent bias
        v = self.V(states).view(-1, 1, 1)
        # Skip connections
        s = 0
        if self.args.skip_connections:
            ws = th.abs(self.skip_connections(states)).view(-1, self.n_agents, 1)
            s = th.bmm(agent_qs, ws)
        # Compute final output
        y = th.bmm(hidden, w_final) + v + s
        # Reshape and return
        q_tot = y.view(bs, -1, 1)
        return q_tot


In [4]:
# load checkpoint mixer

args = pickle.load(open('args.pkl', 'rb'))

mixer = QMixer(args)
mixer.load_state_dict(th.load("{}/{}/mixer.th".format(path, timestep), map_location=lambda storage, loc: storage))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [5]:
# load the episode sample
n_agents = 8
qvals, avail_actions, chosen_qvals, qtots, states, actions = pickle.load(open('info.pkl', 'rb'))

In [6]:
tmp = qvals[0]

min_val = th.min(qvals).item()
max_val = th.max(qvals).item()
print(min_val, max_val)

-0.4408732056617737 0.2584192156791687


In [7]:
# sanity check
length = len(qtots[0])

def get_saliency(i):
    input_qvals = chosen_qvals[:, i,:]
    
    input_state = states[:, i,:]
    output_qtot = mixer(input_qvals, input_state)
    
    input_qvals.retain_grad()
    output_qtot.backward()
    return input_qvals.grad.data[0].numpy()

#for i in range(length):
#    print(get_saliency(i))

In [8]:
def action_string(a_id, action):
    n_actions_no_attack = 6
    act_str = {
        0: "Agent %d: Dead"% a_id,
        1: "Agent %d: Stop"% a_id,
        2: "Agent %d: North"% a_id,
        3: "Agent %d: South"% a_id,
        4: "Agent %d: East"% a_id,
        5: "Agent %d: West"% a_id,
    }
    if action < n_actions_no_attack:
        return act_str[action]
    else:
        target_id = action - n_actions_no_attack
        return "Agent %d attacks enemy # %d" % (a_id, target_id)

In [9]:
import matplotlib as mpl
import matplotlib.cm as cm

cmap = cm.Blues
print(cm)

def get_color(x, vmin=-1, vmax=1):
    m = cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax), cmap=cmap)
    vals = list(m.to_rgba(x)[:3])
    for i in range(len(vals)):
        vals[i] *= 255
    return tuple(vals)

from IPython.display import HTML as html_print

def cstr(s, r=255, g=0, b=0, underline=False):
    u = ""
    if underline:
        return "<text style=background-color:rgb({},{},{})><u>{}</u></text>".format(r,g,b,s)
    else:
        return "<text style=background-color:rgb({},{},{})>{}</text>".format(r,g,b,s)
    
def show_image(num):
    num += 1
    if num < 10:
        num = '0' + str(num)
    #return '3m_screenshots/sc{}.png'.format(num)
    display(Image(filename='screenshots/sc{}.png'.format(num), width=500, unconfined=True))
    
def show_image_html(num):
    num += 1
    if num < 10:
        num = '0' + str(num)
    return "<img src=\"screenshots/sc{}.png\" width=\"500\">".format(num)

def th_html(string):
    return "<th>" + string + "</th>"

def tr_html(string):
    return "<tr>" + string + "</tr>"

def get_title_row():
    actions = ["", "Noop", "Stop", "North", "South", "East", "West"] + ["E" + str(i) for i in range(n_agents)] + ["dQtot/dQ"]
    string = "".join([th_html(a) for a in actions])
    return tr_html(string)

<module 'matplotlib.cm' from '/Users/mika/anaconda3/lib/python3.7/site-packages/matplotlib/cm.py'>


In [10]:
x = ""
for i in range(-20, 20):
    r,g,b = get_color(i, vmin = -20, vmax=20)
    x += cstr("M", r,g,b)
    
html_print(x)

In [11]:
np.set_printoptions(precision=3)
np.set_printoptions(linewidth=np.inf)
%matplotlib inline

qvals_masked = qvals.clone()
qvals_masked[avail_actions == 0] = -np.inf

string = ""
for step in range(length):
    string += " Step {} ".format(step).center(70, '-')
    string += "<table>"
    string += get_title_row()
    saliency_vector = get_saliency(step)
    for a in range(n_agents):
        string += "<tr><th>Q[{}]</th>".format(a)
        for q in qvals_masked[0][step][a].detach().numpy():
            underline = False
            if q == max(qvals_masked[0][step][a].detach().numpy()):
                underline = True
            r,g,b = get_color(q, min_val, max_val)
            val = "{:.3f}".format(q)
            string += "<th>" + cstr(val, r, g, b, underline) + '</th>'
        
        if qvals_masked[0][step][a].detach().numpy()[0] != -np.inf:
            r,g,b = 255,0,0
        else:
            r,g,b = 255,255,255
        string += "<th>" + cstr(" S[{}]={:.2f}".format(a, saliency_vector[a]), r,g,b) + '</th>'
        string += "</tr>"
    string += "</table>"
    string += "<br /> Q_tot = {:.3f}".format(qtots[0][step].detach().numpy()[0])
    string += show_image_html(step)
    
html_print(string)

,Noop,Stop,North,South,East,West,E0,E1,E2,E3,E4,E5,E6,E7,dQtot/dQ
Q[0],-inf,-0.068,0.018,-0.107,-0.108,-0.066,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[0]=1.21
Q[1],-inf,-0.002,-0.072,0.039,-0.026,-0.022,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[1]=1.03
Q[2],-inf,0.012,-0.090,0.069,-0.018,-0.009,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[2]=1.11
Q[3],-inf,-0.030,0.017,-0.071,-0.100,-0.031,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[3]=0.48
Q[4],-inf,-0.032,0.019,-0.077,-0.093,-0.023,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[4]=1.25
Q[5],-inf,0.038,-0.041,0.037,-0.021,0.076,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[5]=1.12
Q[6],-inf,0.025,-0.053,0.049,-0.020,0.030,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[6]=1.44
Q[7],-inf,-0.022,0.045,-0.065,-0.103,-0.002,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[7]=0.97
,Noop,Stop,North,South,East,West,E0,E1,E2,E3,E4,E5,E6,E7,dQtot/dQ
Q[0],-inf,-0.243,-0.164,-0.325,-0.130,-0.356,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,S[0]=0.49
